# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

In [40]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [86]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [87]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [88]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [89]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.427638     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.464235     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [90]:
filter(lambda x :  x['value']>0 , model_all['coefficients'])
    
   

[{'index': None,
  'name': '(intercept)',
  'stderr': None,
  'value': 274873.05595049576},
 {'index': None,
  'name': 'bathrooms',
  'stderr': None,
  'value': 8468.531086910105},
 {'index': None,
  'name': 'sqft_living',
  'stderr': None,
  'value': 24.42072098244546},
 {'index': None,
  'name': 'sqft_living_sqrt',
  'stderr': None,
  'value': 350.06055338605546},
 {'index': None, 'name': 'grade', 'stderr': None, 'value': 842.0680348976231},
 {'index': None,
  'name': 'sqft_above',
  'stderr': None,
  'value': 20.024722417091112}]

In [91]:
model_all["coefficients"][model_all["coefficients"]["value"] > 0]

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


In [37]:
type(model_all['coefficients'])

graphlab.data_structures.sframe.SFrame

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [39]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [94]:
models =[]
predict =[]
for x in np.logspace(1, 7, num=13):
    y = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=x,verbose = False)
    models.append(y)
    x = y.predict(validation)
    predict.append(x)

In [95]:
lowest = 9e15
count = 0
mod = 0
for x in predict:
    count = count +1 
    error = validation['price'] - x
    if np.dot(error,error)<lowest:
        lowest = np.dot(error,error)
        mod = count
print lowest, mod

6.25766285142e+14 1


In [96]:
for x in predict:
    count = count +1 
    error = validation['price'] - x
    print np.dot(error,error)

6.25766285142e+14
6.25766285362e+14
6.25766286058e+14
6.25766288257e+14
6.25766295212e+14
6.25766317206e+14
6.25766386761e+14
6.25766606749e+14
6.25767302792e+14
6.25769507644e+14
6.25776517727e+14
6.25799062845e+14
6.25883719085e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [97]:
filter(lambda x :  x>0 , models[0]['coefficients']['value'])

[18993.4272127706,
 7936.9676790313015,
 936.9933681932994,
 25409.588934120668,
 39.11513637970764,
 1124.650212807717,
 0.003483618222989654,
 148.25839101140826,
 21204.335466950117,
 12915.524336072433,
 601905.594545272,
 93312.85731187189,
 6609.035712447216,
 6206.939991880552,
 43.287053419335614,
 122.3678275341193,
 9.433635393724911,
 56.072003448822365]

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [98]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [111]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [104]:
models =[]
predict =[]
for x in l1_penalty_values:
    y = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=x,verbose = False)
    if y['coefficients']['value'].nnz() >7:
        models.append(x)
    #x = y.predict(validation)
    #predict.append(x)

In [101]:
models

[3792690190.7322536,
 4832930238.5717525,
 6158482110.6602545,
 7847599703.5146227,
 10000000000.0]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [66]:
models =[]
predict =[]
for x in l1_penalty_values:
    y = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=x,verbose = False)
    models.append(y['coefficients']['value'].nnz())
    #if y['coefficients']['value'].nnz() <8:
        #models.append(x)
    #x = y.predict(validation)
    #predict.append(x)

In [67]:
models

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]

In [112]:
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

In [150]:
l1_penalty_min =  2.97635144e+09 #note the above is an array so pick values manually with CARE
l1_penalty_max = 3.79269019e+09

numpy.ndarray

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [151]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
models =[]
predict =[]
abc = []
for x in l1_penalty_values:
    y = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=x,verbose = False)
    models.append(y)
    m = y.predict(validation)
    abc.append((x,y['coefficients']['value'].nnz()))
    predict.append(m)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [152]:
abc

[(2976351440.0, 10),
 (3019316637.3684211, 10),
 (3062281834.7368422, 10),
 (3105247032.1052632, 10),
 (3148212229.4736843, 10),
 (3191177426.8421054, 10),
 (3234142624.2105265, 10),
 (3277107821.5789475, 10),
 (3320073018.9473686, 8),
 (3363038216.3157897, 8),
 (3406003413.6842108, 8),
 (3448968611.0526314, 7),
 (3491933808.4210529, 7),
 (3534899005.7894735, 7),
 (3577864203.1578946, 7),
 (3620829400.5263157, 6),
 (3663794597.8947368, 6),
 (3706759795.2631578, 6),
 (3749724992.6315789, 6),
 (3792690190.0, 6)]

In [157]:
filter(lambda x:  x[1]==7, abc)

[(3448968611.0526314, 7),
 (3491933808.4210529, 7),
 (3534899005.7894735, 7),
 (3577864203.1578946, 7)]

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [155]:
lst =[]
for x in predict:
    count = count +1 
    error = validation['price'] - x
    lst.append(np.dot(error,error))
    print np.dot(error,error)

9.66925692362e+14
9.74019450085e+14
9.81188367942e+14
9.89328342459e+14
9.98783211266e+14
1.00847716702e+15
1.01829878055e+15
1.02824799221e+15
1.03461690923e+15
1.03855473594e+15
1.04323723787e+15
1.04693748875e+15
1.05114762561e+15
1.05599273534e+15
1.06079953176e+15
1.0657076895e+15
1.06946433543e+15
1.07350454959e+15
1.07763277558e+15
1.08186759232e+15


In [159]:
lst[11:15]

[1046937488751710.7,
 1051147625612862.2,
 1055992735342999.2,
 1060799531763287.0]

In [161]:
min(lst[11:15])

1046937488751710.7

In [162]:
filter(lambda x :  x['value']>0 , models[11]['coefficients'])


[{'index': None,
  'name': '(intercept)',
  'stderr': None,
  'value': 222253.1925443279},
 {'index': None,
  'name': 'bedrooms',
  'stderr': None,
  'value': 661.7227177822278},
 {'index': None,
  'name': 'bathrooms',
  'stderr': None,
  'value': 15873.957259268},
 {'index': None,
  'name': 'sqft_living',
  'stderr': None,
  'value': 32.41022145125974},
 {'index': None,
  'name': 'sqft_living_sqrt',
  'stderr': None,
  'value': 690.1147733133166},
 {'index': None, 'name': 'grade', 'stderr': None, 'value': 2899.420269749878},
 {'index': None,
  'name': 'sqft_above',
  'stderr': None,
  'value': 30.011575302201138}]

In [163]:
abc[11]

(3448968611.0526314, 7)